In [1]:
!pip install gradio


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import gradio as gr
import numpy as np
import pickle
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import OneHotEncoder
import os

In [3]:
def load_pickle(filename):
    with open(filename, 'rb') as file:
        data = pickle.load(file)
        return data

pkl_file=load_pickle('modelcomponent1.pkl') 


numeric_imputer=pkl_file['numerical_imputer']
categoric_imputer=pkl_file['categorical_imputer']
scaler_=pkl_file['scaler']
encoder_=pkl_file['encoder']
best_model = pkl_file['model']
top_pack=pkl_file['top_pack_values']

In [4]:
def preprocess_data(Data_Volume, On_Net,Orange, Tigo, Regularity, Freq_Top_Pack,
                                     Region, Tenure, Top_Pack):
    
    df=pd.DataFrame({
        'Data_Volume':[Data_Volume],
        'On_Net':[On_Net],
        'Orange':[Orange],
        'Tigo' :[Tigo],
        'Regularity':[Regularity],
        'Freq_Top_Pack':[Freq_Top_Pack],
        'Region':[Region],
        'Tenure' :[Tenure],
        'Top_Pack':[Top_Pack]
})
    
    # Preprocess the data
    # Selecting categorical and numerical columns separately
    cat_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()
    num_columns = df.select_dtypes(exclude=['object', 'category']).columns.tolist()
    
    

    # Apply the imputers on the input data
    
    cat_imputer=categoric_imputer.transform(df[cat_columns])
    num_imputer=numeric_imputer.transform(df[num_columns])
    
    
    #Encode
    cat_encode = encoder_.transform(cat_imputer)
    cat_encoded_df = pd.DataFrame(cat_encode.toarray(), columns=encoder_.get_feature_names_out(cat_columns))
    print(num_imputer)
    # Scale the numerical columns
    
    num_scaler=scaler_.transform( num_imputer)
    num_scaler_df=pd.DataFrame(num_scaler, columns=num_columns)
    
    #combined
    df_final = pd.concat([num_scaler_df, cat_encoded_df], axis=1)
    #print(df_final)
    return df_final

In [5]:
def predict_churn(Data_Volume, On_Net, Orange,Tigo, Regularity, Freq_Top_Pack, Region, Tenure,  Top_Pack):
  
    preprocessed_data = preprocess_data(Data_Volume, On_Net,Orange, Tigo, Regularity, Freq_Top_Pack, Region, Tenure,  Top_Pack) 
    
    # Make predictions
    predictions = best_model.predict_proba(preprocessed_data)
    #print(predictions)
    return round(predictions[0][0],2), round(predictions[0][1],2)

In [25]:
with gr.Blocks() as demo:
        gr.Markdown('''
        # 📞Telecom Customer Churn Prediction App ☎️ ''') 
        img=gr.Image('churncss.png')
        gr.Markdown('''
        
        ## Welcome Cherished User👋 
        
        ### Please Predict Customer Churn 🙂''')
        with gr.Row():
            Region = gr.Dropdown(label='Region', choices=['DAKAR', 'DIOURBEL', 'FATICK',
                                                  'KAFFRINE', 'KAOLACK', 'KEDOUGOU',
                                                  'KOLDA', 'LOUGA', 'MATAM',
                                                  'SAINT-LOUIS', 'SEDHIOU', 'TAMBACOUNDA', 'THIES', 'ZIGUINCHOR'])
            Tenure = gr.Dropdown(label='Tenure', choices=['K > 24 month', 'I 18-21 month', 'H 15-18 month' , 'G 12-15 month',
                                                  'J 21-24 month','F 9-12 month','E 6-9 month','D 3-6 month'])
        Top_Pack =gr.Dropdown(label='Top_Pack', choices=top_pack)
        with gr.Row():   
            Data_Volume= gr.Number(label='Data_Volume')
            On_Net=gr.Number(label='On_Net')
            
            Orange=gr.Number(label='Orange')
            Tigo=gr.Number(label='Tigo')
            Regularity=gr.Slider(label='Regularity', minimum=int(1), maximum=int(62), value=1, step=1)
            Freq_Top_Pack=gr.Number(label='Freq_Top_Pack')
            
        submit_button=gr.Button('Predict')

        with gr.Row():
            with gr.Accordion('Churn Prediction'):
                output1=gr.Slider(maximum=1,
                             minimum=0,
                             value=1,
                             label='Yes')
                output2=gr.Slider(maximum=1,
                             minimum=0,
                             value=1,
                             label='No')
    
    
    
        submit_button.click(fn=predict_churn,  inputs=[ Data_Volume, On_Net, Orange, Tigo, Regularity, Freq_Top_Pack, Region, Tenure,  Top_Pack ],
                            outputs=[output1, output2])
    
demo.launch()  


Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.
